First we build the Hamiltonian we want in Qiskit

In [32]:
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.synthesis import LieTrotter
from qiskit import transpile
from qiskit_addon_cutting.utils.simulation import ExactSampler

from qiskit_ibm_runtime.options import EstimatorOptions, DynamicalDecouplingOptions
from qiskit_ibm_runtime import EstimatorV2, Batch, SamplerV2
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit_addon_cutting import partition_problem, generate_cutting_experiments, reconstruct_expectation_values

import numpy as np
import json


In [33]:
# Hamiltonian and evolution parameters

num_spins = 6  # Number of qubits per ring
num_rings = 2  # Total number of rings

# Gates that appear in the hamiltonian
two_gates_list = ['ZZ']
single_gates_list = ['X']

num_qubits = num_rings * num_spins
anisotropy = 1.
h = 1.
dt = Parameter('δt')
trotter_reps = 1
dt = 0.1

# Generate edges
edges = [
    (ring * num_spins + i, ring * num_spins + (i + 1) % num_spins)
    for ring in range(num_rings)
    for i in range(num_spins)
]
edges.append((0, 2*num_spins-1))

coupling_map = CouplingMap(edges)

In [34]:
def build_hamiltonian(num_spins, coupling_map, single_gates=['Z'], two_gates=['XX'], anisotropy=1, h=1): 
    edge_list = coupling_map.get_edges()
    hamlist = []
    for edge in edge_list:
        for gate in two_gates:
            hamlist.append((gate, edge, anisotropy))
    for qubit in coupling_map.physical_qubits:
        for gate in single_gates:
            hamlist.append((gate, [qubit], h))
    hamiltonian = SparsePauliOp.from_sparse_list(hamlist, num_qubits=num_spins)
    return hamiltonian


H = build_hamiltonian(num_qubits, coupling_map, single_gates_list, two_gates_list, anisotropy, h)

trotterizator = LieTrotter(reps = trotter_reps, insert_barriers=False)
U = PauliEvolutionGate(operator = H, time = dt)
evolutionQC = trotterizator.synthesize(U)

z_list = [('Z', [i], 1.) for i in range(num_qubits)]
z_observables = SparsePauliOp.from_sparse_list(z_list, num_qubits = num_qubits)


In [35]:
cutting_labels = num_spins * 'A' + num_spins * 'B'
partitioned_problem = partition_problem(circuit = evolutionQC, partition_labels = cutting_labels, observables = z_observables.paulis)
subcircuits = partitioned_problem.subcircuits
subobservables = partitioned_problem.subobservables
bases = partitioned_problem.bases

subexperiments, coefficients = generate_cutting_experiments(circuits = subcircuits, observables = subobservables, num_samples = np.inf)

exact_sampler = ExactSampler()

results = {
    label: exact_sampler.run(subexperiment).result()
    for label, subexperiment in subexperiments.items()
}

reconstructed_expval_terms = reconstruct_expectation_values(results, coefficients, subobservables)
reconstructed_expval = np.dot(reconstructed_expval_terms, z_observables.coeffs)


In [36]:
 
basis_gates = ["h", "rx", "ry", "rz", "rxx", "rzz", "ryy", "cnot"]
readable_data = []

for category in sorted(subexperiments.keys()):
    for index, circuit in enumerate(subexperiments[category]):
        transpiled = transpile(circuit, basis_gates=basis_gates)
        operations = []
        qubit_range = [transpiled.qubits[0]._index, transpiled.qubits[-1]._index]
        
        for instr in transpiled.data:
            name = instr.operation.name
            angle = instr.operation.params
            qubits = [q._index for q in instr.qubits]
            
            if name == 'measure' and (
                not instr.clbits or
                instr.clbits[0]._register.name != 'qpd_measurements'
            ):
                continue
                
            operations.append({
                "Name": name,
                "Angle": angle,
                "Qubits": qubits
            })
        
        readable_data.append({
            "Subexperiment": category,
            "Subcircuit": index,
            "Qubit number": transpiled.num_qubits,
            "Qubit range": qubit_range,
            "Operations": operations
        })


coefficients_list = [coefficients[i][0] for i in range(len(coefficients))]

output_dict = {
    "Expected value": np.real(reconstructed_expval),
    "Coefficients": coefficients_list,
    "Subcircuits": readable_data
}

# Write the structured dictionary to a JSON file
with open("subcircuits.json", "w") as json_file:
    json.dump(output_dict, json_file, indent=4)

print("Subcircuits data has been successfully saved to 'subcircuits.json'.")


Subcircuits data has been successfully saved to 'subcircuits.json'.
